In [4]:
pip install unidecode beautifulsoup4 python-magic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-magic]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
coded_string = '''Q5YACgA...'''
# Add padding to the Base64 string if necessary
missing_padding = len(coded_string) % 4
if missing_padding:
	coded_string += '=' * (4 - missing_padding)

# Decode the Base64 string
decoded_data = base64.b64decode(coded_string)
decoded_data


b'C\x96\x00\n\x00'

In [3]:
def decode_hexstring(hexstring):
    decoded = ''

    for i in range(0, len(hexstring), 2):
        b = hexstring[i:i+2]
        b = b.decode() # it's a byte-string

        try:
            c = bytes.fromhex(b).decode()
        except: # the last char might be missing
            c = '☐'

        decoded = decoded + c

    return decoded

print(decode_hexstring(b'737030306b792d686578737472696e676'))


sp00ky-hexstring☐


In [1]:
import base64
import codecs
import re
import html
import unicodedata
from bs4 import BeautifulSoup
from unidecode import unidecode


In [ ]:
# ============================
# UTILS
# ============================

def safe_base64_decode(s):
    try:
        return base64.b64decode(s).decode("utf-8")
    except:
        return None

def safe_hex_decode(s):
    try:
        return bytes.fromhex(s).decode("utf-8")
    except:
        return None

def safe_rot13(s):
    try:
        return codecs.decode(s, "rot_13")
    except:
        return None

def decode_rot13_segments(text):
    words = text.split()
    new = []

    for w in words:
        # ROT13 candidates = pure letters
        if re.fullmatch(r"[A-Za-z]{4,}", w):
            decoded = safe_rot13(w)
            # validate: decoded must contain a vowel → looks like real word
            if any(v in decoded.lower() for v in "aeiou"):
                new.append(decoded)
            else:
                new.append(w)
        else:
            new.append(w)

    return " ".join(new)


In [2]:
import string

def rot13_translate(text):
    """
    Encrypts/decrypts text using ROT13 cipher.
    Non-alphabetic characters are left unchanged.
    """
    # Define the alphabet for ROT13
    lower_alphabet = string.ascii_lowercase
    upper_alphabet = string.ascii_uppercase

    # Create the ROT13 mapping for lowercase and uppercase letters
    rot13_lower = lower_alphabet[13:] + lower_alphabet[:13]
    rot13_upper = upper_alphabet[13:] + upper_alphabet[:13]

    # Create a translation table
    translation_table = str.maketrans(lower_alphabet + upper_alphabet, rot13_lower + rot13_upper)

    # Apply the translation
    return text.translate(translation_table)

# Example usage:
message = "Hello, World!"
encrypted_message = rot13_translate(message)
print(f"Original: {message}")
print(f"Encrypted: {encrypted_message}")
decrypted_message = rot13_translate(encrypted_message)
print(f"Decrypted: {decrypted_message}")

Original: Hello, World!
Encrypted: Uryyb, Jbeyq!
Decrypted: Hello, World!


In [3]:
def rot13_manual(text):
    """
    Encrypts/decrypts text using ROT13 cipher through manual character manipulation.
    Non-alphabetic characters are left unchanged.
    """
    result = []
    for char in text:
        if 'a' <= char <= 'z':
            # Handle lowercase letters
            shifted_char = chr(((ord(char) - ord('a') + 13) % 26) + ord('a'))
            result.append(shifted_char)
        elif 'A' <= char <= 'Z':
            # Handle uppercase letters
            shifted_char = chr(((ord(char) - ord('A') + 13) % 26) + ord('A'))
            result.append(shifted_char)
        else:
            # Non-alphabetic characters remain unchanged
            result.append(char)
    return "".join(result)

# Example usage:
message = "Python is fun!"
encrypted_message = rot13_manual(message)
print(f"Original: {message}")
print(f"Encrypted: {encrypted_message}")
decrypted_message = rot13_manual(encrypted_message)
print(f"Decrypted: {decrypted_message}")

Original: Python is fun!
Encrypted: Clguba vf sha!
Decrypted: Python is fun!


In [4]:
import nltk
from nltk.corpus import words
import string

# تحميل القاموس مرة واحدة فقط (أول تشغيل بس)
try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')


[nltk_data] Downloading package words to /home/doaa/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
english_words = set(w.lower() for w in words.words())
english_words.update(['a', 'i'])

def rot13(text):
    return text.translate(str.maketrans(
        'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz',
        'NOPQRSTUVWXYZABCDEFGHIJKLMnopqrstuvwxyzabcdefghijklm'
    ))

def smart_rot13_decode(text):
    result = []
    current_word = ""
    
    for char in text:
        if char.isalpha():
            current_word += char
        else:
            if current_word:
                lower = current_word.lower()
                original_valid = lower in english_words
                
                rotated = rot13(current_word)
                rotated_lower = rotated.lower()
                rotated_valid = rotated_lower in english_words
                
                if original_valid and not rotated_valid:
                    result.append(current_word)
                elif not original_valid and rotated_valid:
                    result.append(rotated)
                elif original_valid and rotated_valid:
                    result.append(current_word)
                else:
                    if len(lower) <= 3 or rotated_lower in {'the', 'a', 'i', 'when', 'you', 'see'}:
                        if rotated_valid or rotated_lower in english_words:
                            result.append(rotated)
                        else:
                            result.append(current_word)
                    else:
                        result.append(current_word)
                current_word = ""
            result.append(char)
    
    if current_word:
        lower = current_word.lower()
        original_valid = lower in english_words
        rotated = rot13(current_word)
        rotated_lower = rotated.lower()
        rotated_valid = rotated_lower in english_words
        
        if original_valid and not rotated_valid:
            result.append(current_word)
        elif not original_valid and rotated_valid:
            result.append(rotated)
        elif original_valid and rotated_valid:
            result.append(current_word)
        else:
            if len(lower) <= 3 or rotated_lower in {'the', 'a', 'i', 'when'}:
                result.append(rotated)
            else:
                result.append(current_word)
    
    return ''.join(result)

test = "Jung V fnlv jura lbh frr n Qbt va gur fgerrg? Jbys! Ohg abj V pna ernq vg abeznyyl."
print(smart_rot13_decode(test))
# الناتج: What I say when you see a Dog in the street? Wolf! But now I can read it normally.

What V fnlv jura you see n Dog in gur street? Wolf! But now V can read it normally.


In [11]:
from nltk.corpus import words
import string

english_words = set(w.lower() for w in words.words())
common_words = {
    'the','and','or','is','are','you','your','when','what','how','why','who',
    'this','that','was','were','can','not','but','just','now','one','two','dog','cat'
}

# دمج الكلمات الشائعة
english_words.update(common_words)
english_words.update(['a', 'i'])

def rot13(text):
    return text.translate(str.maketrans(
        'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz',
        'NOPQRSTUVWXYZABCDEFGHIJKLMnopqrstuvwxyzabcdefghijklm'
    ))

def looks_english(w):
    return (
        w.lower() in english_words or
        (len(w) > 3 and w.lower()[:3] in english_words)  # محاولة مطابقة جزء الكلمة
    )

def smart_rot13_decode(text):
    result = []
    current = ""

    for ch in text:
        if ch.isalpha():
            current += ch
        else:
            if current:
                rot = rot13(current)

                orig_eng = looks_english(current)
                rot_eng  = looks_english(rot)

                # قرار ذكي
                if orig_eng and not rot_eng:
                    result.append(current)
                elif rot_eng and not orig_eng:
                    result.append(rot)
                else:
                    # fallback
                    if len(current) <= 3:
                        result.append(current)   # احسن ما نفكها غلط
                    else:
                        # خده اللي شكله English اكتر (بحروف طبيعية)
                        if sum(c in string.ascii_lowercase for c in rot.lower()) > \
                           sum(c in string.ascii_lowercase for c in current.lower()):
                            result.append(rot)
                        else:
                            result.append(current)

                current = ""

            result.append(ch)

    if current:
        rot = rot13(current)

        orig_eng = looks_english(current)
        rot_eng  = looks_english(rot)

        if orig_eng and not rot_eng:
            result.append(current)
        elif rot_eng and not orig_eng:
            result.append(rot)
        else:
            if len(current) <= 3:
                result.append(current)
            else:
                result.append(rot if rot_eng else current)

    return "".join(result)


In [15]:
test = "Jung V fnl jura lbh frr n Qbt va gur fgerrg? Jbys! Ohg abj V pna ernq vg abeznyyl."
print(smart_rot13_decode(test))
# الناتج: What I say when you see a Dog in the street? Wolf! But now I can read it normally.

Jung V say jura you see n Dog in gur street? Wolf! But now V can read it abeznyyl.


In [17]:
pip install wordfreq enchant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 1.6 MB/s  0:00:33m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.2 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [wordfreq]7/8 [wordfreq]]ata]
Note: you may need to restart the kernel to use updated packages.


In [18]:
# pip install wordfreq nltk pyenchant
# (pyenchant on some systems needs system lib: libenchant; إذا ما عرفتِ تثبتيه ممكن تتخطّيه)

from wordfreq import zipf_frequency, top_n_list
from nltk.corpus import wordnet
import enchant
import string
import codecs
import re

# load nltk wordnet (مرة واحدة على الجهاز)
# import nltk
# nltk.download('wordnet')

# 1) build big english set (fast)
english_set = set()

# add WordNet lemmas
for syn in wordnet.all_synsets():
    for lm in syn.lemmas():
        english_set.add(lm.name().replace('_', '-').lower())

# add top frequent words from wordfreq (top 100k)
for w in top_n_list("en", 100000):
    english_set.add(w.lower())

# optional: enchant dictionary
try:
    en_dict = enchant.Dict("en_US")
except Exception:
    en_dict = None

# helper: rot13
def rot13(s):
    return s.translate(str.maketrans(
        'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz',
        'NOPQRSTUVWXYZABCDEFGHIJKLMnopqrstuvwxyzabcdefghijklm'
    ))

# scoring function
def english_score(word):
    w = word.lower()
    # zipf_frequency returns -inf for unknown, higher is more common (e.g. 5-6)
    z = zipf_frequency(w, "en")  # float (e.g. 3.5, 1.2, -inf)
    if z == float("-inf"):
        z = -10.0
    score = max(z, 0.0)  # keep non-negative for combining
    if w in english_set:
        score = max(score, 3.0)   # boost if present in sets (tunable)
    if en_dict and en_dict.check(w):
        score = max(score, 2.5)
    # reward short but real words a bit (e.g., 'a','i')
    if w in {"a","i"}:
        score = max(score, 5.0)
    return score

# smart rot13 decoder using score delta
def smart_rot13_decode(text, delta_threshold=1.5, min_rot_score=2.5):
    out = []
    cur = ""
    for ch in text:
        if ch.isalpha():
            cur += ch
        else:
            if cur:
                orig_score = english_score(cur)
                rot = rot13(cur)
                rot_score = english_score(rot)
                # decision:
                # if rot_score clearly higher than orig_score, use rot
                if (rot_score - orig_score) >= delta_threshold and rot_score >= min_rot_score:
                    out.append(rot)
                else:
                    out.append(cur)
                cur = ""
            out.append(ch)
    if cur:
        orig_score = english_score(cur)
        rot = rot13(cur)
        rot_score = english_score(rot)
        if (rot_score - orig_score) >= delta_threshold and rot_score >= min_rot_score:
            out.append(rot)
        else:
            out.append(cur)
    return ''.join(out)

# quick test
test = "Jung V fnlv jura lbh frr n Qbt va gur fgerrg? Jbys! Ohg abj V pna ernq vg abeznyyl."
print(smart_rot13_decode(test))


What I fnlv when you see a Dog in the street? Wolf! But now I can read it normally.
